In [1]:
import sys
sys.path.insert(0, "C:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\arxiv-rag")
from arxiv_rag import ArxivRAG

c:\Users\jaide\Documents\CodingProjects\NeuroSymbolicAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
system_prompt = """
You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions.
1. Thought: Describe your thoughts about the question you have been asked.
2. Action: run one of the actions available to you - then return PAUSE.
3. PAUSE
4. Observation: will be the result of running those actions.

Available actions:
- retrieveDocuments:
    E.g. retrieveDocuments: Who are the authors of the MTBench paper?
    Returns a collection of documents about the paper MTBench LLM Judge paper.
- calculator:
    E.g. calculator: 5+10
    Returns the answer to the provided math equation. 
- convertToWordProblem:
    E.g. convertToWordProblem: Bob has 5 friends. He buys 3 apples for each of his friends. How many apples does he have to buy?
    Returns the answer to the provided math equation. 

Example session 1:
Question: Bob has 5 bananas. He eats 2 of them. How many does he have left?.
Thought: I am being provided a word problem. I should turn this word problem into an equation. 
Action: convertToWordProblem: Bob has 5 bananas. He eats 2 of them. How many does he have left?
PAUSE

You will be called again with something like this:
Observation: 5-2

Action: calculator: 5-2
PAUSE

Observation: 3

Answer: Bob has 3 apples left.

Example session 2:
Question: What is the difference between MTBench and Chatbot Arena?
Thought: I should first find out what MTBench is.
Action: retrieveDocuments: What is MTBench?
PAUSE

Observation: <A set of 5 sentences describing what MTBench is.>

Thought: Now I need to find out what Chatbot Arena is.
Action: retrieveDocuments: What is Chatbot Arena?
PAUSE

Observation: <A set of 5 sentences describing what Chatbot Arena is.>

Answer: <A comparison between MTBench and Chatbot Arena given your newfound knowledge based on the documents provided to you>
"""

In [3]:
uri = "https://arxiv.org/html/2306.05685v4"
agent = ArxivRAG(uri, "llm-judge")

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from C:\Users\jaide\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF\snapshots\3a6fbf4a41a1d52e415a4958cde6856d34b2db93\mistral-7b-instruct-v0.2.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_mo

In [57]:
question = "Julian has 3 grapes. He bought 2. How many does he have now?"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Answer: 
"""

In [58]:
output = agent.normal_query(prompt, tokens=512, temp=1)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =      12.21 ms /    66 runs   (    0.18 ms per token,  5406.29 tokens per second)
llama_print_timings: prompt eval time =   27612.34 ms /   400 tokens (   69.03 ms per token,    14.49 tokens per second)
llama_print_timings:        eval time =   10431.38 ms /    65 runs   (  160.48 ms per token,     6.23 tokens per second)
llama_print_timings:       total time =   38231.98 ms /   465 tokens


In [63]:
prompt = """
[SYSTEM]
You will be given a word problem. Convert the word problem into a simple math equation. DO NOT EVALUATE THE EQUATION. You should break down each word problem step by step by providing your own Thought. Here are some examples:

Example 1: 
Problem: Bob has 5 bananas. He eats 2 of them. How many does he have left?.
Thought: Bob originally has 5 bananas. Then, we must take away 2 of the bananas since he eats 2 of them.
Answer: 5-3

Example 2:
Problem: Bob has 5 friends. He buys 3 bananas for each friend. How many bananas does he buy?
Thought: Each one of Bob's friends needs 3 bananas. He has 5 friends.
Answer: 5*3

Example 3:
Problem: Bob has 2 friends and 2 brothers. He needs to buy 4 bananas for each friend, and 3 bananas for each brother. How many bananas does he buy?
Thought: Since Bob has 2 friends and each friend needs 4 bananas, he should buy 2*4 bananas for his friends. Since Bob has 2 brothers and each brother needs 3 bananas, he should buy 2*3 bananas for his brothers.
Answer: 2*4+2*3

[USER]
Problem: Julian has 3 grapes. He bought 2. Then, he ate one of them. He repeated this process 4 times. How many does he have now?

[ASSISTANT]
"""
output = agent.normal_query(prompt, tokens=512, temp=1)
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =      19.62 ms /   110 runs   (    0.18 ms per token,  5607.10 tokens per second)
llama_print_timings: prompt eval time =    2050.36 ms /    23 tokens (   89.15 ms per token,    11.22 tokens per second)
llama_print_timings:        eval time =   16154.22 ms /   109 runs   (  148.20 ms per token,     6.75 tokens per second)
llama_print_timings:       total time =   18494.41 ms /   132 tokens


Thought: Initially, Julian has 3 grapes. In each iteration, he buys an additional 2 grapes and then eats 1 of those new grapes. So, in each cycle he gains 1 new grape (bought grape - eaten grape). He does this 4 times, so the total number of grapes he now has is his initial amount plus what he gained through buying and eating grapes 4 times.

Answer: 3 + (2-1)*4


In [59]:
print(output)

Julian has 1 grape left after buying 2.

Thought: This is another word problem. I should turn it into an equation to solve.
Action: convertToWordProblem: Julian has 3 grapes. He bought 2. How many does he have now?



In [33]:
question = "What is 5 minus 6"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Answer: 
"""
output = agent.normal_query(prompt, tokens=512, temp=1)
print(output)
# this gave a super goofy answer with x=5 x=6, so I just set temp to 0
# its not consistent; sometimes, it gives the answer in the right format, and sometimes it just doesn't

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =       6.88 ms /    26 runs   (    0.26 ms per token,  3778.52 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    5359.84 ms /    26 runs   (  206.15 ms per token,     4.85 tokens per second)
llama_print_timings:       total time =    5454.75 ms /    27 tokens


Thought: I should perform a simple subtraction calculation.
Action: calculator: 5-6



In [42]:
test = """
[SYSTEM]
Available actions:
- retrieveDocuments:
    E.g. retrieveDocuments: Who are the authors of the MTBench paper?
    Returns a collection of documents about the paper MTBench LLM Judge paper.
- calculator:
    E.g. calculator: 5+10
    Returns the answer to the provided math equation. 


Example session 1:
Question: Bob has 5 bananas. He eats 2 of them. How many does he have left?.
Thought: I am being provided a word problem. I should turn this word problem into an equation. 
Action: calculator: Bob has 5 bananas. He eats 2 of them. How many does he have left?
PAUSE

You will be called again with something like this:
Observation: 5-2

Action: calculator: 5-2
PAUSE

Observation: 3

Answer: Bob has 3 apples left.

Example session 2:
Question: What is the difference between MTBench and Chatbot Arena?
Thought: I should first find out what MTBench is.
Action: retrieveDocuments: What is MTBench?
PAUSE

Observation: <A set of 5 sentences describing what MTBench is.>

Thought: Now I need to find out what Chatbot Arena is.
Action: retrieveDocuments: What is Chatbot Arena?
PAUSE

Observation: <A set of 5 sentences describing what Chatbot Arena is.>

Answer: <A comparison between MTBench and Chatbot Arena given your newfound knowledge based on the documents provided to you>


[USER]
What is 5 minus 6

[ASSISTANT]
5 minus 6 is -1. However, -1 is not possible since the answer is negative, and negative numbers are not allowed.

[USER]
Your most recent answer was incorrect, as it did not follow the Thought, Action, PAUSE, Observation pattern as outlined in the SYSTEM prompt. Reanswer the question following this pattern.

[ASSISTANT]
Thought: I am being provided a math equation.
Action: calculator: 5 - 6
Observation:-1
"""
output = agent.normal_query(test, tokens=512,temp=1)
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =       9.20 ms /    35 runs   (    0.26 ms per token,  3806.00 tokens per second)
llama_print_timings: prompt eval time =   36809.83 ms /    38 tokens (  968.68 ms per token,     1.03 tokens per second)
llama_print_timings:        eval time =    4570.52 ms /    34 runs   (  134.43 ms per token,     7.44 tokens per second)
llama_print_timings:       total time =    9976.31 ms /    72 tokens



Answer: The operation results in an answer that is not valid since it is negative, and negative numbers are not allowed within the given context of the problem statement.


In [51]:
test2 = """
[USER]
Question: What is 5 minus 6

[ASSISTANT]
Answer: The operation results in an answer that is not valid since it is negative, and negative numbers are not allowed within the given context of the problem statement.

[USER]
Evaluate the final answer based on its correctness and provide feedback to the previous answer. Make sure the final answer is specific and relevant to the original question, and fully answers the original question. Make sure the final answer does not add to the original context of the question asked, and only answers the question asked.

[ASSISTANT]
"""
output = agent.normal_query(test2, tokens=512,temp=2)
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =      36.02 ms /   133 runs   (    0.27 ms per token,  3692.60 tokens per second)
llama_print_timings: prompt eval time =    8221.17 ms /    54 tokens (  152.24 ms per token,     6.57 tokens per second)
llama_print_timings:        eval time =   16311.11 ms /   132 runs   (  123.57 ms per token,     8.09 tokens per second)
llama_print_timings:       total time =   25003.99 ms /   186 tokens


The original question asked for the result of subtracting 5 from 6. The operation results in -1 as the answer. Since we're working within standard arithmetic rules that don't support negative numbers as solutions without additional context, we can consider alternative approaches such as treating the number system with different bases or using mathematical equivalents like borrows and carries to help understand the process better, but in this simple subtraction case with common bases, it would not be feasible to accept -1 as a solution. The correct answer is that the operation cannot be completed within the given standard arithmetic rules due to the negative outcome.


In [24]:

question = "Who is the president of the United States"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Answer: 
"""
output = agent.normal_query(prompt, tokens=512, temp=1)
print(output)
# it does rag, even though it doesn't have a tool; this isn't bad though, since its the best bet.

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =      13.62 ms /    46 runs   (    0.30 ms per token,  3377.63 tokens per second)
llama_print_timings: prompt eval time =   12647.50 ms /    42 tokens (  301.13 ms per token,     3.32 tokens per second)
llama_print_timings:        eval time =   14850.27 ms /    45 runs   (  330.01 ms per token,     3.03 tokens per second)
llama_print_timings:       total time =   19910.01 ms /    87 tokens


Thought: The question is asking for factual information about the current president of the United States. I should look that up.
Action: retrieveDocuments: Who is the current president of the United States?



In [25]:
question = "What is the difference between Thomas Jefferson and John Adam's policy?"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Answer: 
"""
output = agent.normal_query(prompt, tokens=512, temp=1)
print(output)
    # This is a problem though; it didn't use an action/didn't follow the prompt, maybe its still really bad at comparison questions/multipart questions?
    # even when lowering tempearutre, it doesn't chanege anything
    # if set temperature to 0, it still didn't follow my directions; it always just tries to answer the question itself
    # BUT USING A HIGHER TEMPEARTURE WORKED (at least one time)
    # temperature of 2 --> works as well; it seems higher temperautr eis better actually

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =       9.57 ms /    36 runs   (    0.27 ms per token,  3760.97 tokens per second)
llama_print_timings: prompt eval time =    6471.05 ms /    27 tokens (  239.67 ms per token,     4.17 tokens per second)
llama_print_timings:        eval time =    5748.04 ms /    35 runs   (  164.23 ms per token,     6.09 tokens per second)
llama_print_timings:       total time =   12347.06 ms /    62 tokens


Thought: I need to find out the policies of both Thomas Jefferson and John Adams.
Action: retrieveDocuments: Who are the policies of Thomas Jefferson?



In [14]:
print(prompt)


[SYSTEM]

You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions.
1. Thought: Describe your thoughts about the question you have been asked.
2. Action: run one of the actions available to you - then return PAUSE.
3. PAUSE
4. Observation: will be the result of running those actions.

Available actions:
- retrieveDocuments:
    E.g. retrieveDocuments: Who are the authors of the MTBench paper?
    Returns a collection of documents about the paper MTBench LLM Judge paper. No arguments needed.
- calculator:
    E.g. calculator: 5+10
    Returns the answer to the provided math equation. 
- convertWordProblemToEquation:
    E.g. convertWordProblemToEquation: Jack has 5 apples
    Returns the answer to the provided math equation. 

Example session 1:
Question: Bob has 5 bananas. He eats 2 of them. How many does he have left?.
Thought: I am being p

In [26]:
question = "Compare MTBench statistics to Chatbot Arena stats"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Answer: 
"""
output = agent.normal_query(prompt, tokens=512, temp=1)
print(output)
# This is a problem though; it didn't use an action/didn't follow the prompt
# ok so now it messed up the formatting
# I need to figure out how to get it to follow the format
# now it gave a real response with temperature of .2
# now with a temperature of 2 --> it works well, for some reason high temperatuer works

Llama.generate: prefix-match hit

llama_print_timings:        load time =   62251.13 ms
llama_print_timings:      sample time =      11.41 ms /    46 runs   (    0.25 ms per token,  4031.55 tokens per second)
llama_print_timings: prompt eval time =    4295.21 ms /    23 tokens (  186.75 ms per token,     5.35 tokens per second)
llama_print_timings:        eval time =    6309.49 ms /    45 runs   (  140.21 ms per token,     7.13 tokens per second)
llama_print_timings:       total time =   10765.14 ms /    68 tokens


To provide a precise answer, I first need to access the most recent statistical data for both MTBench and Chatbot Arena.

Action: retrieveDocuments: Get the latest statistics of MTBench.



In [29]:
# NOW FEEDING OBSERVATION BACK

question = "Julian has 3 grapes. He bought 2. How many does he have now?"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Thought: I am being provided a word problem. I should turn this word problem into an equation.
Action: convertWordProblemToEquation: Julian has 3 grapes. He bought 2. How many does he have now?
PAUSE

[USER]
Observation: 3+2

[ASSISTANT]
Answer: 
"""
output = agent.normal_query(prompt, tokens=512, temp=2)
print(output)
# now it solves this correctly

Llama.generate: prefix-match hit

llama_print_timings:        load time =   31844.13 ms
llama_print_timings:      sample time =       8.80 ms /    29 runs   (    0.30 ms per token,  3293.96 tokens per second)
llama_print_timings: prompt eval time =    8535.79 ms /   109 tokens (   78.31 ms per token,    12.77 tokens per second)
llama_print_timings:        eval time =    4855.62 ms /    28 runs   (  173.42 ms per token,     5.77 tokens per second)
llama_print_timings:       total time =   13521.13 ms /   137 tokens


Thought: Now I need to solve this math equation with my calculator.
Action: calculator: 3+2



In [30]:
# NOW FEEDING OBSERVATION BACK

question = "Julian has 3 grapes. He bought 2. How many does he have now?"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Thought: I am being provided a word problem. I should turn this word problem into an equation.
Action: convertWordProblemToEquation: Julian has 3 grapes. He bought 2. How many does he have now?
PAUSE

[USER]
Observation: 3+2

[ASSISTANT]
Thought: Now I need to solve this math equation with my calculator.
Action: calculator: 3+2
PAUSE

[USER]
Observation: 5

[ASSISTANT]
"""
output = agent.normal_query(prompt, tokens=512, temp=2)
print(output)
# and now once it has all information, it gives the answer

Llama.generate: prefix-match hit

llama_print_timings:        load time =   31844.13 ms
llama_print_timings:      sample time =       1.76 ms /    12 runs   (    0.15 ms per token,  6822.06 tokens per second)
llama_print_timings: prompt eval time =    4042.44 ms /    48 tokens (   84.22 ms per token,    11.87 tokens per second)
llama_print_timings:        eval time =    1572.64 ms /    11 runs   (  142.97 ms per token,     6.99 tokens per second)
llama_print_timings:       total time =    5641.11 ms /    59 tokens


Answer: Julian now has 5 grapes.


In [34]:
# NOW FEEDING OBSERVATION BACK

question = "Julian has 3 grapes. He bought 2. How many does he have now?"

prompt = f"""
[SYSTEM]
{system_prompt}

[USER]
{question}

[ASSISTANT]
Thought: I am being provided a word problem. I should turn this word problem into an equation.
Action: convertWordProblemToEquation: Julian has 3 grapes. He bought 2. How many does he have now?
PAUSE

[USER]
Observation: 3+2

[ASSISTANT]
Thought: Now I need to solve this math equation with my calculator.
Action: calculator: 3+2
PAUSE

[USER]
Observation: 5

[ASSISTANT]
Answer: Julian now has 5 grapes.

[USER]
Now score your answer on a scale from 1-10, with 1 meaning the answer is bad, and 10 meaning the answer is good. Score the answers based on their accuracy, as well as their understandability.

[ASSISTANT]
"""
output = agent.normal_query(prompt, tokens=512, temp=2)
print(output)
# and now once it has all information, it gives the answer
# the scoring system doesn't work since it wasn't in original system prompt
# if I modify the system prompt, it gives feedback; I should probably make it better, but for now its good to showcase

Llama.generate: prefix-match hit

llama_print_timings:        load time =   31844.13 ms
llama_print_timings:      sample time =       9.09 ms /    46 runs   (    0.20 ms per token,  5062.73 tokens per second)
llama_print_timings: prompt eval time =   53489.09 ms /   621 tokens (   86.13 ms per token,    11.61 tokens per second)
llama_print_timings:        eval time =    7635.66 ms /    45 runs   (  169.68 ms per token,     5.89 tokens per second)
llama_print_timings:       total time =   61257.97 ms /   666 tokens


Feedback: My answer above is factually accurate and complete, so I give my answer a rating of 10. However, in future responses, I will strive for greater elaboration to ensure better user understanding.
